<a href="https://colab.research.google.com/github/samaksh97/DVA/blob/main/financial_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sec-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Api key : a7810c91f294b072f631b43305a6c16b37a1b5708e6ad97e2ae04f3908a98fb9

In [2]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="a7810c91f294b072f631b43305a6c16b37a1b5708e6ad97e2ae04f3908a98fb9")

In [3]:
query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" AND ticker:TSLA", # only 10-Ks
  }},
  "from": "0", # start returning matches from position null, i.e. the first matching filing 
  "size": "1"  # return just one filing
}

response = queryApi.get_filings(query)

In [4]:
import json 
print(json.dumps(response["filings"][0], indent=2))

{
  "ticker": "TSLA",
  "formType": "10-K",
  "accessionNo": "0001193125-14-069681",
  "cik": "1318605",
  "companyNameLong": "TESLA MOTORS INC (Filer)",
  "companyName": "TESLA MOTORS INC",
  "linkToFilingDetails": "https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/d668062d10k.htm",
  "description": "Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]",
  "linkToTxt": "https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/0001193125-14-069681.txt",
  "filedAt": "2014-02-26T16:02:51-05:00",
  "documentFormatFiles": [
    {
      "sequence": "1",
      "size": "1589148",
      "documentUrl": "https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/d668062d10k.htm",
      "description": "10-K",
      "type": "10-K"
    },
    {
      "sequence": "2",
      "size": "71602",
      "documentUrl": "https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/d668062dex1035a.htm",
      "description": "EX-10.35A",
      "type": "EX

In [5]:
base_query = {
  "query": { 
      "query_string": { 
          "query": "PLACEHOLDER", # this will be set during runtime 
          "time_zone": "America/New_York"
      } 
  },
  "from": "0",
  "size": "200", # dont change this
  # sort returned filings by the filedAt key/value
  "sort": [{ "filedAt": { "order": "desc" } }]
}

In [7]:

# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "a")

# start with filings filed in 2022, then 2020, 2019, ... up to 1995
# uncomment next line to fetch all filings filed from 2022-1995
# for year in range(2021, 1994, -1):
for year in range(2022, 2020, -1):
  print("Starting download for year {year}".format(year=year))
  
  # a single search universe is represented as a month of the given year
  for month in range(1, 13, 1):
    # get 10-Q and 10-Q/A filings filed in year and month
    # resulting query example: "formType:\"10-Q\" AND filedAt:[2021-01-01 TO 2021-01-31]"
    universe_query = \
        "formType:(\"10-K\", \"10-KT\", \"10KSB\", \"10KT405\", \"10KSB40\", \"10-K405\") AND " + \
        "filedAt:[{year}-{month:02d}-01 TO {year}-{month:02d}-31]" \
        .format(year=year, month=month)
  
    # set new query universe for year-month combination
    base_query["query"]["query_string"]["query"] = universe_query;

    # paginate through results by increasing "from" parameter 
    # until we don't find any matches anymore
    # uncomment next line to fetch all 10,000 filings
    # for from_batch in range(0, 9800, 200): 
    for from_batch in range(0, 400, 200):
      # set new "from" starting position of search 
      base_query["from"] = from_batch;

      response = queryApi.get_filings(base_query)

      # no more filings in search universe
      if len(response["filings"]) == 0:
        break;

      # for each filing, only save the URL pointing to the filing itself 
      # and ignore all other data. 
      # the URL is set in the dict key "linkToFilingDetails"
      urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))

      # transform list of URLs into one string by joining all list elements
      # and add a new-line character between each element.
      urls_string = "\n".join(urls_list) + "\n"
      
      log_file.write(urls_string)

    print("Filing URLs downloaded for {year}-{month:02d}".format(year=year, month=month))

log_file.close()

print("All URLs downloaded")


Starting download for year 2022
Filing URLs downloaded for 2022-01
Filing URLs downloaded for 2022-02
Filing URLs downloaded for 2022-03
Filing URLs downloaded for 2022-04
Filing URLs downloaded for 2022-05
Filing URLs downloaded for 2022-06
Filing URLs downloaded for 2022-07
Filing URLs downloaded for 2022-08
Filing URLs downloaded for 2022-09
Filing URLs downloaded for 2022-10
Filing URLs downloaded for 2022-11
Filing URLs downloaded for 2022-12
Starting download for year 2021
Filing URLs downloaded for 2021-01
Filing URLs downloaded for 2021-02
Filing URLs downloaded for 2021-03
Filing URLs downloaded for 2021-04
Filing URLs downloaded for 2021-05
Filing URLs downloaded for 2021-06
Filing URLs downloaded for 2021-07
Filing URLs downloaded for 2021-08
Filing URLs downloaded for 2021-09
Filing URLs downloaded for 2021-10
Filing URLs downloaded for 2021-11
Filing URLs downloaded for 2021-12
All URLs downloaded


In [8]:
from sec_api import RenderApi

renderApi = RenderApi(api_key="a7810c91f294b072f631b43305a6c16b37a1b5708e6ad97e2ae04f3908a98fb9")

In [9]:
# download filing and save to "filings" folder
def download_filing(url):
  try:
    filing = renderApi.get_filing(url)
    # file_name example: 000156459019027952-msft-10k_20190630.htm
    file_name = url.split("/")[-2] + "-" + url.split("/")[-1] 
    download_to = "./filings/" + file_name
    with open(download_to, "w") as f:
      f.write(filing)
  except Exception as e:
    print("Problem with {url}".format(url=url))
    print(e)

In [10]:
def load_urls():
  log_file = open("filing_urls.txt", "r")
  urls = log_file.read().split("\n") # convert long string of URLs into a list 
  log_file.close()
  return urls

In [12]:
import os
import multiprocessing

def download_all_filings():
  print("Start downloading all filings")

  download_folder = "./filings" 
  if not os.path.isdir(download_folder):
    os.makedirs(download_folder)
    
  # uncomment next line to process all URLs
  # urls = load_urls()
  urls = load_urls()[1:40]
  print("{length} filing URLs loaded".format(length=len(urls)))

  number_of_processes = 20

  with multiprocessing.Pool(number_of_processes) as pool:
    pool.map(download_filing, urls)
  
  print("All filings downloaded")

In [13]:
download_all_filings()

Start downloading all filings
39 filing URLs loaded
All filings downloaded
